<a href="https://colab.research.google.com/github/EricJoung1997/plotlyDash-test/blob/main/%E4%B8%BB%E9%A2%98%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import jieba
from collections import Counter
from tqdm import tqdm
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
import re

# 加载数据

In [2]:
cnki = pd.read_excel('/content/drive/MyDrive/ColabNotebooks/刘博文/CNKI-2.xlsx')

wos = pd.read_excel('/content/drive/MyDrive/ColabNotebooks/刘博文/WoS-2.xlsx')

# 1. CNKI聚类

## 1.1 中文预处理

In [3]:

my_stopwords =  [i.strip() for i in open('/content/drive/MyDrive/ColabNotebooks/刘博文/stop_word.txt',encoding='utf-8').readlines()]
#去停用词+分词
cnki['cut'] = cnki['Summary-摘要'].apply(lambda x : ' '.join([j.strip() for j in jieba.lcut(x) if j not in my_stopwords]))


'''
这里我又增加了一些停用词
例如方法、目的、结论、结果、分析、意义、创新

'''

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 1.098 seconds.
DEBUG:jieba:Loading model cost 1.098 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [7]:
#分词结果示例
cnki['cut'].sample(5)

3763    甘肃省 哲学 社会科学 优秀 成果奖 我省 哲学 社会科学 领域 最高 奖项 采用 互联网 ...
299     认知 神经科学 领域 脑电 技术 为例 利用 时序 纵向 探索 知识 交叉 路径 借助 共词...
856     知识 单元 游离 组合 知识 计量 理论 基础 时间 加权 修正 后 提取 关键词 知识 单...
259     旨在 揭示 固体 氧化物 燃料电池 SOFC 技术 历史 全貌 深入 挖掘 全球 各国 合作...
3205    国内 编辑出版 类 作者 选题 同行 编辑 组稿 选稿 提供 一定 参考 依据 以期 编辑出...
Name: cut, dtype: object

## 1.2 执行聚类

In [15]:

#加载预训练模型，使用bert-base-chinese中文模型为CNKI数据进行语义化编码
embeddings_cnki = SentenceTransformer('bert-base-chinese',device='cuda')

In [16]:
topic_model_cnki = BERTopic(
                            language="chinese (simplified)", #语言为中文
                            verbose=True,         # 模型详细程度，TRUE or FALSE
                            embedding_model=embeddings_cnki, #预训练模型
                            calculate_probabilities=True, #是否输出概率
                            top_n_words=5,      #每个主题的关键词数
                          )

#聚类拟合
topics_cnki = topic_model_cnki.fit(cnki['cut'].tolist())

Batches:   0%|          | 0/119 [00:00<?, ?it/s]

2023-09-01 03:37:05,629 - BERTopic - Transformed documents to Embeddings
2023-09-01 03:37:33,958 - BERTopic - Reduced dimensionality
2023-09-01 03:37:34,468 - BERTopic - Clustered reduced embeddings


## 1.3 聚类结果

In [24]:
'''
列的含义：
Topic（主题编号），-1表示废弃的主题，不需要
Count（主题下的文章数），
Name（简易的主题名称），
Representation（主题下的主题词），
Representative_Docs（代表性的文章）
'''
topics_cnki.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1240,-1_文献_领域_主题_知识,"[文献, 领域, 主题, 知识, 数据]",[农业 采用 领域 知识 演化 历程 趋势 可视化 中国 知网 数据库 WoS 核心 数据库...
1,0,700,0_情报_竞争_情报学_数据,"[情报, 竞争, 情报学, 数据, 安全]",[大 数据 环境 下 情报 流程 探讨 情报工作 面临 机遇 挑战 过程 提出 以大 数据 ...
2,1,519,1_期刊_评价_指标_影响力,"[期刊, 评价, 指标, 影响力, 影响]",[图书馆学 情报学 期刊 综合 质量 评价 发现 样本 期刊 质量 评价 年间 变化 期刊 ...
3,2,155,2_知识_关联_主题_语义,"[知识, 关联, 主题, 语义, 图谱]",[系统 梳理 科研 关系 网络 理论 构建 科研 关系 网络 构建 基础 上 知识 关联 服...
4,3,127,3_治理_热点_中国_文献,"[治理, 热点, 中国, 文献, 城市]",[聚集 特定 城市 区域 产业 地球表层 人类 利用 资源 环境 技术 进步 产物 经济学 ...
5,4,113,4_文献_分布_热点_领域,"[文献, 分布, 热点, 领域, 数据库]",[CNKI CSSCI 数据库 收录 1994 以来 国内 古籍整理 保护 领域 核心 期刊...
6,5,92,5_学科_交叉_跨学科_知识,"[学科, 交叉, 跨学科, 知识, 交流]",[文章 情报学 计算机 跨学科 应用 为例 学科 交叉 领域 基础 学科 交叉 关联 基础 ...
7,6,88,6_技术_识别_颠覆性_新兴,"[技术, 识别, 颠覆性, 新兴, 识别方法]",[颠覆性 技术 技术 市场 具有 双重 影响 无论是 科技 市场 结构调整 都 起到 十分 ...
8,7,86,7_教育_学习_教师_教学,"[教育, 学习, 教师, 教学, 素养]",[教育 信息化 飞速发展 新形势下 教育 技术 学 现状 值得 研究者 关注 问题 文章 采...
9,8,70,8_专利_技术_价值_专利权人,"[专利, 技术, 价值, 专利权人, 企业]",[专利 引文 评价 当前 专利 情报分析 手段 基本原理 依据 某项 专利 专利 引用 频次...


In [29]:
# 结果存为excel
topics_cnki.get_topic_info().to_excel('/content/drive/MyDrive/ColabNotebooks/刘博文/CNKI总体聚类.xlsx')

## 1.4 可视化

In [25]:
#每个主题下的主题词可视化

topic_model_cnki.visualize_barchart(top_n_topics=len(topics_cnki.get_topic_info()))

In [28]:
#可视化主题、主题大小、距离及其对应的单词
topic_model_cnki.visualize_topics()

# 2. WoS聚类

## 2.1 数据预处理

In [56]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
# 下载停用词
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [58]:
#只保留摘要不为空的
wos = wos[wos['Abstract'].notnull()]

# 加载停用词
stop_words_en = set(stopwords.words('english'))
# 初始化词形还原器
lemmatizer = WordNetLemmatizer()

# 定义停用词移除和词形还原函数
def remove_stopwords_lemmatize(text):
    # 分词
    tokens = word_tokenize(text)
    # 去除停用词和词形还原
    filtered_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words_en]
    # 连接成字符串
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

# 在 "cut" 列上应用停用词移除函数
wos['cut'] = wos['Abstract'].apply(remove_stopwords_lemmatize)

In [59]:
wos['cut']

0       innovation literature often argues major inven...
1       currently prevailing international ranking sys...
2       although prior study stressed importance techn...
3       objective : qualitative research examines tran...
4       technology significantly changed way human liv...
                              ...                        
5636    objective : study sought conduct informatics a...
5637    regular physical activity help prevent heart d...
5638    report action research investigation knowledge...
5639    business value information technology ( ) one ...
5640    modern information seeker face dynamic stream ...
Name: cut, Length: 5627, dtype: object

## 2.2 执行聚类

In [60]:

#加载预训练模型，使用all-MiniLM-L6-v2模型为WoS数据进行语义化编码
embeddings_wos = SentenceTransformer('all-MiniLM-L6-v2',device='cuda')

In [61]:
topic_model_wos = BERTopic(
                            language="english", #语言为英文
                            verbose=True,         # 模型详细程度，TRUE or FALSE
                            embedding_model=embeddings_wos, #预训练模型
                            calculate_probabilities=True, #是否输出概率
                            top_n_words=5,      #每个主题的关键词数
                          )

#聚类拟合
topics_wos = topic_model_wos.fit(wos['cut'].tolist())

Batches:   0%|          | 0/176 [00:00<?, ?it/s]

2023-09-01 04:39:07,634 - BERTopic - Transformed documents to Embeddings
2023-09-01 04:39:19,758 - BERTopic - Reduced dimensionality
2023-09-01 04:39:21,646 - BERTopic - Clustered reduced embeddings


## 2.3 聚类结果

In [62]:
'''
列的含义：
Topic（主题编号），-1表示废弃的主题，不需要
Count（主题下的文章数），
Name（简易的主题名称），
Representation（主题下的主题词），
Representative_Docs（代表性的文章）
'''
topics_wos.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2125,-1_research_science_citation_paper,"[research, science, citation, paper, information]",[scientific journal currently primary medium u...
1,0,652,0_health_patient_care_ehr,"[health, patient, care, ehr, clinical]",[objective : describe implementation technolog...
2,1,287,1_user_intention_perceived_social,"[user, intention, perceived, social, consumer]",[purpose number smartphone user increased matu...
3,2,168,2_patent_technology_technological_convergence,"[patent, technology, technological, convergenc...",[forward citation patent used extensively capt...
4,3,159,3_ict_broadband_economic_growth,"[ict, broadband, economic, growth, mobile]","[paper , dynamic relationship ict , income ine..."
...,...,...,...,...,...
71,70,11,70_acquisition_acquirer_merger_integration,"[acquisition, acquirer, merger, integration, t...",[study examines impact coordination capability...
72,71,11,71_recommendation_recommender_preference_listener,"[recommendation, recommender, preference, list...",[article identifies factor impact mobile recom...
73,72,11,72_relation_thesaurus_document_structure,"[relation, thesaurus, document, structure, ent...",[continuing growth scientific publication pose...
74,73,10,73_path_mpa_main_network,"[path, mpa, main, network, citation]",[main path analysis useful tool form backbone ...


In [63]:
# 结果存为excel
topics_wos.get_topic_info().to_excel('/content/drive/MyDrive/ColabNotebooks/刘博文/WoS总体聚类.xlsx')

## 2.4 可视化

In [64]:
#每个主题下的主题词可视化

topic_model_wos.visualize_barchart(top_n_topics=len(topics_wos.get_topic_info()))

In [65]:
#可视化主题、主题大小、距离及其对应的单词
topic_model_wos.visualize_topics()

# WoS按国家聚类

In [71]:
#只保留摘要不为空的
wos = wos[wos['Reprint Addresses'].notnull()]

In [97]:
#字符串清洗，例如OR 97201 USA替换为USA
def replace_usa_string(text):
    pattern = re.compile(r'.*USA.*')
    replaced_text = re.sub(pattern, 'USA', text)
    return replaced_text

#解析国家名称
def praser_country(text):
  country = text.split(', ')[-1].replace(".", "")
  country_re = replace_usa_string(country)
  return country_re

wos['country'] = wos['Reprint Addresses'].apply(praser_country)

In [102]:
pd.DataFrame(Counter(wos['country']).most_common(),columns=['country','count']).to_excel('/content/drive/MyDrive/ColabNotebooks/刘博文/WoS国家论文数.xlsx')




In [94]:
# Counter(wos['country_re']).most_common()

replace_usa_string('China USA')

'USA'